# Data Science Capstone project notebook: Part 1
Autor: Sebastien SIME
_____

## Table of Contents

1.  <a href="##1">Libraries and installations</a>
2.  <a href="#item1">Canada neighborhood Web-Scrapping using Pandas and BeautifulSoup modules</a>
3.  <a href="#item1">Creation of the final data frame</a>

## 1. Libraries and installations

First the neede libraries:

In [1]:
# Data Science lybraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# for HTML page
from bs4 import BeautifulSoup

Seocndly, the url that will be used for the following operations:

In [2]:
# Webpage url                                                                                                               
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

## 2.  Web-scrapping:

### Using pandas

In [3]:
# Extract tables
data = pd.read_html(url)

In [4]:
# Number of tables:
print(len(data))

3


In [5]:
# Store the tables                                                                                                           
df = data[0]
df1 = data[1]
df2 = data[2]

In [6]:
# Table 1:
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Let's try another technic.

### Using beatiful soup

In [7]:
import requests
req = requests.get(url)
#
soup = BeautifulSoup(req.text, "html.parser")
#print(soup.title)

In [8]:
# A view of all the page:
#print(soup.prettify()) # to see all the html tags

The table is identified with the following tag:

In [9]:
# To find a table
table = soup.table
#table

In [10]:
# to find the columns headers i.e the columns' names
col = table.find_all('th')
col = [i.text.rstrip() for i in col]
#col

In [11]:
# To select all the rows
table_rows = table.find_all('tr')
#table_rows

In [12]:
# to store all the rows in a list and print it:
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    #print(row)

In [13]:
# a test to remove "\n"
row[1].rstrip()

'Not assigned'

In [14]:
# To store all the rows in a single list
rows = []
for tr in table_rows:
    td = tr.find_all('td')
    rows.append( [i.text.rstrip() for i in td] )
    #print(rows)

In [15]:
len(rows)

181

## 3. Data frame creation:

All the above operations effectivelly lead to a data frame with the relevant table from Wikipedia. I found that the **read_html** function is more efficient for the task. I created a function to ease the process of parsing with beautifulSoup below:

In [16]:
# Automation: to parse a table using beutifulSoup module
def getTable(url):
    # Libraries:
    from bs4 import BeautifulSoup
    import requests
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")

    # To find a table
    table = soup.table # select the table in the html
    table_rows = table.find_all('tr') # select all the row (tr r for row)
    table_columns = [i.text.rstrip() for i in table.find_all('th')] # select all the header of each columns
    
    # Built a list of lists with all the data
    rows = []
    for tr in table_rows:
        td = tr.find_all('td') # select all the data
        rows.append( [i.text.rstrip() for i in td] ) # extract the text from the data
    
    # return the data frame:
    df = pd.DataFrame(rows, columns = table_columns)
    return (df)
#

In [17]:
# call of the above function
Toronto_neighbourhood = getTable(url)

In [18]:
Toronto_neighbourhood.head()

,Postal Code,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [19]:
# Since the first row is empty, we can drop the latter
Toronto_neighbourhood.drop(Toronto_neighbourhood.index[0], inplace = True)

In [20]:
# Let's reset the indexes
Toronto_neighbourhood = Toronto_neighbourhood.reset_index()

In [21]:
# Drop the newly created index column as well
Toronto_neighbourhood.drop(columns = 'index', inplace = True)

### Data Frame cleaning:

In [22]:
Toronto_neighbourhood.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [23]:
Toronto_neighbourhood['Borough'].value_counts() # 77 boroughs Not assigned

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [24]:
#1 Only process the cells that have an assigned borough:
Toronto_neighbourhood.drop(Toronto_neighbourhood[Toronto_neighbourhood["Borough"] == "Not assigned"].index, inplace = True)
Toronto_neighbourhood.reset_index(inplace = True)
Toronto_neighbourhood.head()

,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [25]:
Toronto_neighbourhood.shape

(103, 4)

In [26]:
#2 Combine Borough with same Postal Code: OK
len(Toronto_neighbourhood['Postal Code'].unique())

103

In [27]:
Toronto_neighbourhood['Postal Code'].duplicated().sum()

0

There is no duplicate rows in the 'Postal Code' column.

In [28]:
#3 Not assigned Neighborhood:
len(Toronto_neighbourhood.loc[Toronto_neighbourhood['Neighbourhood'] == "Not assigned", 'Neighbourhood'])
#Toronto_neighborhood['Neighbourhood'].isnull().sum()

0

All the rows in the column 'Neighborhood' are filled.

In [29]:
#4 Final Cleaning: 'index' column drop
Toronto_neighbourhood.drop(columns = 'index', inplace = True)
Toronto_neighbourhood.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [30]:
# 'Postal Code' column renamed:
Toronto_neighbourhood.rename(columns={'Postal Code': 'PostalCode'}, inplace = True)
Toronto_neighbourhood.head(12)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [31]:
# Shape of the final data frame:
Toronto_neighbourhood.shape

(103, 3)

In [38]:
# Export fichier
Toronto_neighbourhood.to_csv('Toronto_neighbourhood.csv')
print('file exported')

file exported
